# 01 - Preparação dos dados

Neste notebook busca-se apenas estruturar o diretório dos dados para treinamento.

Como primeiro passo, vamos baixar o arquivo `production-line-classification-dataset.zip` (disponível no [Kaggle](https://www.kaggle.com/datasets/christianvorhemus/industrial-quality-control-of-packages) no diretório atual de trabalho vamos transferi-lo para o diretório `../transfer` dentro do Colab.

In [1]:
! mkdir -p ../transfer
! curl -L -o ../transfer/industrial-quality-control-of-packages.zip https://www.kaggle.com/api/v1/datasets/download/christianvorhemus/industrial-quality-control-of-packages
! unzip -oq ../transfer/industrial-quality-control-of-packages.zip -d ../transfer/raw-data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  231M  100  231M    0     0  62.1M      0  0:00:03  0:00:03 --:--:-- 78.2M


## Importações

In [2]:
import os
import shutil
import glob
import numpy as np
from google.colab import files

## Constantes e sets

In [3]:
DATA_DIR = '../data'  # Ajustar conforme ambiente de execução
SEED = 42  # Para garantir reprodutibilidade
TRAIN_PROP = 0.7

np.random.seed(SEED)

## Scripts

Para garantir que está tudo certo, antes de mais nada vamos limpar (caso exista) os diretórios que serão criados.

In [4]:
%%capture

! rm -r {DATA_DIR} {DATA_DIR}-tmp

In [5]:
%%capture

! mv ../transfer/raw-data {DATA_DIR}-tmp

In [6]:
%%capture

! apt update
! apt install tree -y

In [7]:
! tree {DATA_DIR}-tmp -aL 2

../data-tmp
├── damaged
│   ├── side
│   └── top
└── intact
    ├── side
    └── top

6 directories, 0 files


In [8]:
dmg_ids = [f.split('_')[0] for f in os.listdir(os.path.join(f'{DATA_DIR}-tmp', 'damaged', 'side'))]  # Todos que estão em side estão em top
itc_ids = [f.split('_')[0] for f in os.listdir(os.path.join(f'{DATA_DIR}-tmp', 'intact', 'side'))]

train_dmg_ids = np.random.choice(dmg_ids, size=int(TRAIN_PROP*len(dmg_ids)), replace=False)
train_itc_ids = np.random.choice(itc_ids, size=int(TRAIN_PROP*len(itc_ids)), replace=False)

train_dmg_files = [file for dmg_id in train_dmg_ids for file in glob.glob(os.path.join(f'{DATA_DIR}-tmp', f'**/{dmg_id}_*.png'), recursive=True)]
train_itc_files = [file for itc_id in train_itc_ids for file in glob.glob(os.path.join(f'{DATA_DIR}-tmp', f'**/{itc_id}_*.png'), recursive=True)]

len(train_dmg_files + train_itc_files)

280

Com isso, vamos criar o diretório que vamos de fato armazenar os dados, fazendo a divisão entre treino e teste, junto as classes.

In [9]:
os.makedirs(os.path.join(DATA_DIR, 'train'), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, 'train', 'damaged'), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, 'train', 'intact'), exist_ok=True)

for file in train_dmg_files:
    shutil.move(file, os.path.join(DATA_DIR, 'train', 'damaged'))

for file in train_itc_files:
    shutil.move(file, os.path.join(DATA_DIR, 'train', 'intact'))

os.makedirs(os.path.join(DATA_DIR, 'test'), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, 'test', 'damaged'), exist_ok=True)
os.makedirs(os.path.join(DATA_DIR, 'test', 'intact'), exist_ok=True)

test_dmg_files = glob.glob(os.path.join(f'{DATA_DIR}-tmp', 'damaged/**/*_*.png'), recursive=True)  # Tudo o que sobrou
test_itc_files = glob.glob(os.path.join(f'{DATA_DIR}-tmp', 'intact/**/*_*.png'), recursive=True)

for file in test_dmg_files:
    shutil.move(file, os.path.join(DATA_DIR, 'test', 'damaged'))

for file in test_itc_files:
    shutil.move(file, os.path.join(DATA_DIR, 'test', 'intact'))

len(glob.glob(os.path.join(DATA_DIR, 'train', '**/*_*.png'), recursive=True)), len(glob.glob(os.path.join(DATA_DIR, 'test', '**/*_*.png'), recursive=True))

(280, 120)

Agora vamos remover o diretório temporário e comprimir os dados para utilizá-los em outros notebooks no colab. Os dados serão salvos em `../transfer/data.zip`.

In [10]:
%%capture

! rm -r {DATA_DIR}-tmp
! zip -ro ../transfer/data.zip {DATA_DIR}

E para facilitar, vamos baixar o arquivo final (caso esteja rodando no Colab).

In [11]:
files.download('../transfer/data.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>